In [1]:
import os

In [2]:
import import_ipynb
import aux.relation_extraction

importing Jupyter notebook from /Users/YK/mt/project/aux/relation_extraction.ipynb


In [3]:
%run explanation_01.ipynb
%run explanation_02.ipynb
%run explanation_03.ipynb
%run explanation_04.ipynb
%run explanation_05.ipynb
# %run explanation_06.ipynb
%run explanation_07.ipynb
# %run explanation_08.ipynb
# %run topic_comment_01.ipynb

importing Jupyter notebook from /Users/YK/mt/project/aux/utils.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/nlp.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/defs.ipynb
importing Jupyter notebook from preparation.ipynb
importing Jupyter notebook from rule_base.ipynb


In [4]:
rules = [
    RuleExplanation01(),
    RuleExplanation02(),
    RuleExplanation03(),
    RuleExplanation04(),
    RuleExplanation05(),
#     RuleExplanation06(),
    RuleExplanation07(),
#     RuleExplanation08(),
#     RuleTopicComment01()
]

In [10]:
def create_directories(statements_directory, dataset_subdirectory, rules):
    output_directories = {}
    for rule in rules:
        rule_output_directory = os.path.join(
            statements_directory, rule.name, dataset_subdirectory
        )
        os.makedirs(rule_output_directory, exist_ok=True)
        output_directories[rule.name] = rule_output_directory
    return output_directories


def generate_statements(
    dataset_directory,
    dataset_subdirectory,
    statements_directory,
    rules
):
    output_directories = create_directories(
        statements_directory, dataset_subdirectory, rules
    )
    
    input_directory = os.path.join(dataset_directory, dataset_subdirectory)
    file_names = os.listdir(input_directory)
    for i, file_name in enumerate(file_names):
        print(f"[{i + 1:4d}/{len(file_names)}, {file_name}]")
        
        text, relations, cleaned_tree_text = aux.relation_extraction.load_relations(
            os.path.join(input_directory, file_name)
        )
        
        if "Topic-Comment" in relations:
            root = aux.relation_extraction.read_relation_tree(cleaned_tree_text)
            assert root is not None
            _, main_nucleus_segment = root.get_first_nucleus()
            main_nucleus_text = text[
                main_nucleus_segment.start:main_nucleus_segment.end
            ]
        else:
            main_nucleus_text = None
    
        for rule in rules:
            print(f"  -- {rule.name}")
            if rule.relation_type in relations:
                statements = []
                for relation in relations[rule.relation_type]:
                    statement = rule.generate_statement(
                        text,
                        relation,
                        main_nucleus_text=main_nucleus_text
                    )
                    if statement is not None:
                        statements.append(statement._asdict())
                if len(statements) > 0:
                    with open(
                        os.path.join(output_directories[rule.name], file_name), "wt"
                    ) as f:
                        f.write(json.dumps(statements, indent=2))

In [ ]:
generate_statements(
    "../parsed/race/",#directory
    "train/middle",#subdirectory
    "statements_3",#statements directory
    rules
)

[3216/6386, 6287.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3217/6386, 1120.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3218/6386, 4380.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3219/6386, 3241.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3220/6386, 7229.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3221/6386, 2089.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3222/6386, 4529.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explan

  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3273/6386, 1681.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3274/6386, 3970.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3275/6386, 4421.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3276/6386, 4647.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3277/6386, 1877.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3278/6386, 7588.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3279/6386, 2728.t

  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3330/6386, 4560.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3331/6386, 6601.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3332/6386, 3831.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3333/6386, 7858.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3334/6386, 2668.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3335/6386, 4561.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3336/6386, 6600.t

  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3387/6386, 2506.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3388/6386, 425.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3389/6386, 5857.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3390/6386, 2996.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3391/6386, 643.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3392/6386, 7950.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3393/6386, 2760.txt

  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3443/6386, 5311.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3444/6386, 7070.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3445/6386, 7216.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3446/6386, 5177.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3447/6386, 3019.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3448/6386, 194.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[3449/6386, 7217.tx

...

  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4345/6386, 445.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4346/6386, 2566.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4347/6386, 4232.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4348/6386, 1092.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4349/6386, 6153.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4350/6386, 6335.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4351/6386, 4054.tx

  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4401/6386, 7997.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4402/6386, 5666.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4403/6386, 7507.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4404/6386, 684.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4405/6386, 1469.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4406/6386, 3708.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4407/6386, 6938.tx

  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4457/6386, 4937.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4458/6386, 3466.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4459/6386, 5509.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4460/6386, 2858.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4461/6386, 5999.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4462/6386, 8076.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4463/6386, 6209.t

  -- explanation_04
  -- explanation_05
  -- explanation_07
[4513/6386, 2412.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4514/6386, 2883.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4515/6386, 530.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4516/6386, 5942.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4517/6386, 2413.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4518/6386, 2675.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4519/6386, 756.txt.tree]
  -- explanation_01
  -- explanat

  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4569/6386, 438.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4570/6386, 6715.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4571/6386, 3925.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4572/6386, 4474.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4573/6386, 4612.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4574/6386, 1822.txt.tree]
  -- explanation_01
  -- explanation_02
  -- explanation_03
  -- explanation_04
  -- explanation_05
  -- explanation_07
[4575/6386, 6573.txt.tree]
  -- explana